In [2]:
import numpy as np

In [3]:
list1 = list(np.random.randint(1,10000,500))
list2 = list1 + [list1[2]] * 5
list3 = list1.copy()
list3.append(-3)

lists = [list1,list2,list3]

In [124]:
def sort_list(long_list):
    new_list = []
    for element in long_list:
        new_list = sort_element(new_list, element)
    return new_list

def sort_element(mini_list, element, idx=0, how=None):

    if idx < 0:
        return [element] + mini_list
    
    elif idx == len(mini_list):
        return mini_list + [element]

    elif element < mini_list[idx]:
        if how == 'high':
            return mini_list[:idx] + [element] + mini_list[idx:]

        idx -= 1
        return sort_element(mini_list, element, idx, how='low')
    
    elif element == mini_list[idx]:
        return mini_list[:idx] + [element] + mini_list[idx:]
    
    elif element > mini_list[idx]:
        if how == 'low':
            return mini_list[:idx+1] + [element] + mini_list[idx+1:]

        idx += 1
        return sort_element(mini_list, element, idx, how='high')



In [125]:
def bubble_sort(nums):
    for _ in range(len(nums)-1):
        for i in range(len(nums)-1):

            if nums[i] > nums[i+1]:
                nums[i], nums[i+1] = nums[i+1], nums[i]
    
    return nums

In [5]:
def insertion_sort(nums):
    nums = list(nums)
    for i in range(len(nums)):
        current = nums.pop(i)
        j = i -1
        while j>=0 and nums[j]>current:
            j -= 1
        nums.insert(j+1, current)
    return nums

In [4]:
def merge_sort(nums):
    if len(nums) <= 1:
        return nums
    
    mid = len(nums) // 2

    left = nums[:mid]
    right = nums[mid:]

    left_sorted, right_sorted = merge_sort(left), merge_sort(right)

    sorted_nums = merge(left_sorted, right_sorted)
    return sorted_nums

def merge(nums1, nums2):

    merged = []
    i, j = 0, 0

    while i < len(nums1) and j < len(nums2):

        if nums1[i] <= nums2[j]:
            merged.append(nums1[i])
            i += 1
        else:
            merged.append(nums2[j])
            j += 1
        
    nums1_tail = nums1[i:]
    nums2_tail = nums2[j:]

    return merged + nums1_tail + nums2_tail

In [17]:
def quicksort(nums, start=0, end=None):

    if end is None:
        nums = list(nums)
        end = len(nums) - 1
    
    if start < end:
        pivot = partition(nums, start, end)
        quicksort(nums, start, pivot-1)
        quicksort(nums, pivot+1, end)

    return nums

def partition(nums, start=0, end=None):
    if end is None:
        end = len(nums) - 1

    l, r = start, end-1

    while r > l:
        
        if nums[l] <= nums[end]:
            l += 1
        elif nums[r] > nums[end]:
            r -= 1
        else:
            nums[r], nums[l] = nums[l], nums[r]

    if nums[l] > nums[end]:
        nums[l], nums[end] = nums[end], nums[l]
        return l
    else:
        return end

In [18]:
list1 = list(np.random.randint(1,10000,25000))

In [19]:
%%timeit

quicksort(list1)

104 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit

merge_sort(list1)

130 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
